In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.datasets import mnist
import keras

In [106]:
# Generate the train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
 
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
 
# Generate some noisy data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
 
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [107]:
def make_model():
    model = Sequential()

    model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

def train_model(model, x_train, y_train):
    epochs = 45
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+epochs))
    model.fit(x_train, y_train, batch_size=64, epochs = epochs, steps_per_epoch = x_train.shape[0]//64, callbacks=[annealer])

In [108]:
model = make_model()
model.save('base_model')

INFO:tensorflow:Assets written to: base_model/assets


In [109]:
m1 = tf.keras.models.load_model('base_model')
train_model(m1, x_train, y_train)

Epoch 1/45
937/937 [==============================] - 99s 106ms/step - loss: 0.7787 - accuracy: 0.7607
Epoch 2/45
937/937 [==============================] - 107s 114ms/step - loss: 0.1907 - accuracy: 0.9422
Epoch 3/45
937/937 [==============================] - 115s 123ms/step - loss: 0.1187 - accuracy: 0.9645
Epoch 4/45
937/937 [==============================] - 101s 108ms/step - loss: 0.0873 - accuracy: 0.9737
Epoch 5/45
937/937 [==============================] - 105s 112ms/step - loss: 0.0709 - accuracy: 0.9785
Epoch 6/45
937/937 [==============================] - 105s 112ms/step - loss: 0.0569 - accuracy: 0.9831
Epoch 7/45
937/937 [==============================] - 102s 109ms/step - loss: 0.0507 - accuracy: 0.9851
Epoch 8/45
937/937 [==============================] - 103s 110ms/step - loss: 0.0447 - accuracy: 0.9864
Epoch 9/45
937/937 [==============================] - 104s 110ms/step - loss: 0.0407 - accuracy: 0.9878
Epoch 10/45
937/937 [==============================] - 106s 113ms

In [110]:
m1.save('based_model')

INFO:tensorflow:Assets written to: based_model/assets


In [111]:
results = m1.evaluate(x_test, y_test)
results = m1.evaluate(x_test_noisy, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 7.3875 - accuracy: 0.1158


In [112]:
m2 = tf.keras.models.load_model('based_model')
train_model(m2, x_train_noisy[:30000], y_train[:30000])

Epoch 1/45
468/468 [==============================] - 43s 92ms/step - loss: 0.2437 - accuracy: 0.9297
Epoch 2/45
468/468 [==============================] - 43s 92ms/step - loss: 0.1480 - accuracy: 0.9534
Epoch 3/45
468/468 [==============================] - 43s 92ms/step - loss: 0.1215 - accuracy: 0.9615
Epoch 4/45
468/468 [==============================] - 43s 92ms/step - loss: 0.1080 - accuracy: 0.9659
Epoch 5/45
468/468 [==============================] - 43s 92ms/step - loss: 0.1021 - accuracy: 0.9673
Epoch 6/45
468/468 [==============================] - 43s 92ms/step - loss: 0.0909 - accuracy: 0.9700
Epoch 7/45
468/468 [==============================] - 43s 92ms/step - loss: 0.0877 - accuracy: 0.9707
Epoch 8/45
468/468 [==============================] - 43s 92ms/step - loss: 0.0791 - accuracy: 0.9747
Epoch 9/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0786 - accuracy: 0.9745
Epoch 10/45
468/468 [==============================] - 43s 91ms/step - loss: 0.071

In [113]:
results = m2.evaluate(x_test, y_test)
results = m2.evaluate(x_test_noisy, y_test)

313/313 [==============================] - 3s 11ms/step - loss: 0.0724 - accuracy: 0.9794


In [114]:
m2.save('sudden_drift_30000')

INFO:tensorflow:Assets written to: sudden_drift_30000/assets


In [115]:
m3 = tf.keras.models.load_model('base_model')
train_model(m3, np.vstack((x_train, x_train_noisy[:30000])), np.vstack((y_train, y_train[:30000])))

Epoch 1/45
1406/1406 [==============================] - 130s 92ms/step - loss: 0.7953 - accuracy: 0.7538
Epoch 2/45
1406/1406 [==============================] - 131s 93ms/step - loss: 0.2226 - accuracy: 0.9302
Epoch 3/45
1406/1406 [==============================] - 131s 93ms/step - loss: 0.1455 - accuracy: 0.9550
Epoch 4/45
1406/1406 [==============================] - 131s 93ms/step - loss: 0.1109 - accuracy: 0.9657
Epoch 5/45
1406/1406 [==============================] - 131s 93ms/step - loss: 0.0933 - accuracy: 0.9716
Epoch 6/45
1406/1406 [==============================] - 131s 93ms/step - loss: 0.0806 - accuracy: 0.9750
Epoch 7/45
1406/1406 [==============================] - 130s 93ms/step - loss: 0.0697 - accuracy: 0.9788
Epoch 8/45
1406/1406 [==============================] - 131s 93ms/step - loss: 0.0641 - accuracy: 0.9802
Epoch 9/45
1406/1406 [==============================] - 129s 92ms/step - loss: 0.0587 - accuracy: 0.9818
Epoch 10/45
1406/1406 [==============================] 

In [116]:
m3.save('new_trained')

INFO:tensorflow:Assets written to: new_trained/assets


In [117]:
k1 = tf.keras.models.load_model('based_model')
train_model(m2, x_train[:30000], y_train[:30000])

Epoch 1/45
468/468 [==============================] - 42s 91ms/step - loss: 0.0180 - accuracy: 0.9943
Epoch 2/45
468/468 [==============================] - 42s 91ms/step - loss: 0.0152 - accuracy: 0.9955
Epoch 3/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0139 - accuracy: 0.9955
Epoch 4/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0123 - accuracy: 0.9960
Epoch 5/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0126 - accuracy: 0.9958
Epoch 6/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0105 - accuracy: 0.9966
Epoch 7/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0106 - accuracy: 0.9966
Epoch 8/45
468/468 [==============================] - 43s 92ms/step - loss: 0.0097 - accuracy: 0.9966
Epoch 9/45
468/468 [==============================] - 43s 91ms/step - loss: 0.0085 - accuracy: 0.9975
Epoch 10/45
468/468 [==============================] - 43s 91ms/step - loss: 0.008

In [118]:
k1.save("half_normal")
results = k1.evaluate(x_test, y_test)
results = k1.evaluate(x_test_noisy, y_test)

INFO:tensorflow:Assets written to: half_normal/assets
313/313 [==============================] - 3s 10ms/step - loss: 7.3875 - accuracy: 0.1158


In [119]:
x_gc_train = np.vstack((x_train[30000:35000], x_train_noisy[:5000]))
y_gc_train = np.vstack((y_train[30000:35000], y_train[:5000]))

idx = [i for i in range(len(x_gc_train))]
np.random.shuffle(idx)

x_gc_train = x_gc_train[idx]
y_gc_train = y_gc_train[idx]

x_gc_train.shape
y_gc_train.shape

(10000, 10)

In [120]:
train_model(k1, x_gc_train, y_gc_train)

Epoch 1/45
156/156 [==============================] - 14s 90ms/step - loss: 0.2241 - accuracy: 0.9329
Epoch 2/45
156/156 [==============================] - 14s 90ms/step - loss: 0.1129 - accuracy: 0.9648
Epoch 3/45
156/156 [==============================] - 14s 90ms/step - loss: 0.0949 - accuracy: 0.9699
Epoch 4/45
156/156 [==============================] - 14s 90ms/step - loss: 0.0805 - accuracy: 0.9721
Epoch 5/45
156/156 [==============================] - 14s 90ms/step - loss: 0.0669 - accuracy: 0.9801
Epoch 6/45
156/156 [==============================] - 14s 90ms/step - loss: 0.0674 - accuracy: 0.9770
Epoch 7/45
156/156 [==============================] - 14s 90ms/step - loss: 0.0622 - accuracy: 0.9786
Epoch 8/45
156/156 [==============================] - 14s 89ms/step - loss: 0.0564 - accuracy: 0.9814
Epoch 9/45
156/156 [==============================] - 14s 90ms/step - loss: 0.0522 - accuracy: 0.9814
Epoch 10/45
156/156 [==============================] - 14s 90ms/step - loss: 0.050

In [121]:
k1.save("half_normal_added_10k_mix")
results = k1.evaluate(x_test, y_test)
results = k1.evaluate(x_test_noisy, y_test)

INFO:tensorflow:Assets written to: half_normal_added_10k_mix/assets
313/313 [==============================] - 3s 10ms/step - loss: 0.1092 - accuracy: 0.9710


In [78]:
# Check how averaging layer 6 weights affects performance

k1 = tf.keras.models.load_model('half_normal')
k2 = tf.keras.models.load_model('half_normal_added_10k_mix')

w1, b1 = k1.layers[6].get_weights()
w2, b2 = k2.layers[6].get_weights()

w3 = (w1+w2)/2

k1.layers[6].set_weights([w3, b1])
k2.layers[6].set_weights([w3, b2])

results = k1.evaluate(x_test, y_test)
results = k1.evaluate(x_test_noisy, y_test)

results = k2.evaluate(x_test, y_test)
results = k2.evaluate(x_test_noisy, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.1425 - accuracy: 0.9672


In [94]:
from keras.models import Model
import tensorflow as tf
import numpy as np

def get_fisher_info(model, raw_data, label_data):
    y = model.output

    row_idx = tf.range(tf.shape(y)[0])
    col_idx = tf.argmax(y, axis=1, output_type=tf.dtypes.int32)
    full_indices = tf.stack([row_idx, col_idx], axis=1)
    fx_tensors = tf.gather_nd(y, full_indices)

    x_tensors = model.trainable_weights

    num_samples = 100
    m = Model(inputs=model.input, outputs=fx_tensors)

    fisher_information = []
    for v in range(len(x_tensors)):
        fisher_information.append(np.zeros(x_tensors[v].get_shape().as_list()).astype(np.float32))

    for i in range(num_samples):
        data_idx = np.random.randint(raw_data.shape[0])
        sampled_data = raw_data[data_idx:data_idx+1]
        sampled_input_variables = [ sampled_data ]
#         print ('sample num: %4d, data_idx: %5d' % (i, data_idx))

        with tf.GradientTape() as tape:
            p = m(sampled_data)
            lo = tf.math.log(p)

        gradients = tape.gradient(lo, x_tensors)
        derivatives = [g.numpy() for g in gradients]
        prob = p.numpy()[0]
        
    #     derivatives, prob = sess.run([tf.gradients(tf.log(fx_tensors), x_tensors), fx_tensors],
    #     feed_dict={t: v for t,v in zip(input_tensors, sampled_input_variables)})

        for v in range(len(fisher_information)):
            fisher_information[v] += np.square(derivatives[v]) * prob
        
    return fisher_information

In [95]:
k1 = tf.keras.models.load_model('half_normal')
k2 = tf.keras.models.load_model('half_normal_added_10k_mix')

f1 = get_fisher_info(k1, x_train[:30000], y_train[:30000])
f2 = get_fisher_info(k1, x_gc_train, y_gc_train)

In [96]:
%matplotlib widget

for i, arr in enumerate(f1):
    plt.plot(arr.flatten(), label='layer {}'.format(i))
plt.ylabel('some numbers')
plt.legend(loc="upper right")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
%matplotlib widget

for i, arr in enumerate(f2):
    plt.plot(arr.flatten(), label='layer {}'.format(i))
plt.ylabel('some numbers')
plt.legend(loc="upper right")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …